In [ ]:
import json
import xmltodict
import boto3


#Input name of bucket containing data (sample_script.fdx)

BUCKET_NAME = ""

#Input name of endpoint hosting the BlazingText Model. 
ENDPOINT_NAME = ""

s3_client = boto3.client('s3')
sagemaker_client = boto3.client('sagemaker-runtime')

In [ ]:
#Method to extract fdx scripts from s3, loads them as dict item. 

def read_script_from_s3(key: str, client=s3_client) -> dict:
    response = client.get_object(Bucket=BUCKET_NAME, Key=key)
    content = response['Body'].read().decode('utf-8')
    return xmltodict.parse(content)

#Method to send/receive requests to the active endpoint. 

def query_endpoint(text_input: str, client=sagemaker_client):
    response = sagemaker_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, Body=f'{{"instances\": ["{text_input}"]}}')
    return json.loads(response["Body"].read().decode())

In [ ]:
#Load the script, extract the text content.

script = read_script_from_s3("data/sample_script.fdx")
script_content = script['FinalDraft']['Content']['Paragraph']

In [ ]:
#Test endpoint with a negative review.
query_endpoint("The worst guitar strings, very bad, don't buy.")

In [ ]:
#Test endpoint with a positive review.
query_endpoint("This is a great keyboard, keys are high quality!") 

In [ ]:
#Test endpoint on a negative sentiment line from the script.
print(script_content[379]['Text'])
query_endpoint(script_content[379]['Text'])

In [ ]:
#Test endpoint on a positive sentiment line from the script.
print(script_content[46]['Text'])
query_endpoint(script_content[46]['Text'])

In [ ]:
#Find all lines from the script with the highest positive sentiment. 

for i in range(0, len(script_content)):
    if 'Text' in script_content[i]:
        result = query_endpoint(script_content[i]['Text'])
        if result[0]['label'][0] == "__label__5":
            print(i, ": ", script_content[i]['Text'])